In [35]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd

In [36]:
df_TOCC = pd.read_csv('../../datasets/TypeOfContractChannel.csv')
df_TOCC[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


### X 범주형 , Y 범주형 : 주로 독립성 증명 
- 귀무가설 : 관계성이 없다
- 대립가설 : 관계성이 있다.
    - RF) 대립가설은 연구원이 증명하고자 하는 가설로써 증명하고자 하는 가설이다.

#### 선정 변수 : type_of_contract, cancellation
- 분석 내용 : 계약 유형에 따라 해약여부가 달라지는가?
    - 귀무가설 : 계약 유형에 따라 해약여부가 달라지지 않는다.
    - 대립가설 : 계약 유형에 따라 해약여부가 달라진다.
- 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
    - 통계 결론 : 대립가설 참, 귀무가설 기각
    - 사실 결론 : 계약 유형에 따라 해약여부가 달라진다. P.value < 0.05 : 대립 참

In [37]:
df_TOCC[['type_of_contract', 'cancellation']].describe()

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [38]:
df_TOCC_category = df_TOCC[['type_of_contract', 'cancellation']]

In [39]:
df_TOCC_category_pivot = pd.pivot_table(data=df_TOCC_category, index='type_of_contract' , columns='cancellation', aggfunc=len) # aggfunc=len -> count
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


#### p-value 값을 확인하는 이유 
- 숫자로만은 비교가 되지 않아 %도 비교하기 위함 
- 평균을 동일하게 하여 그래프를 겹치게해서 양 끝쪽의 차가 p-value
- 범주/범주형의 비교를 분포도로 본다.

In [40]:
import scipy
scipy.__version__
import scipy.stats as stats

In [41]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True) #data는 집계된 data로, correction는 자유도

Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

#### 선정 변수 : product, cancellation
- 분석 내용 : 차종에 따라 해약여부가 달라지는가?
    - 귀무가설 : 차종에 따라 해약여부가 달라지지 않는다.
    - 대립가설 : 차종에 따라 해약여부가 달라진다.
- 분석결론 (기준:p-value 0.05)
    - 통계 결론 : 대립가설
    - 사실 결론 :  차종에 따라 해약여부가 달라진다. P.value < 0.05 : 대립 참

In [42]:
# 다른 변수 이름 작성 시 copy() 사용
df_TOCC_product = df_TOCC[['product', 'cancellation']].copy()
df_TOCC_product[:2]

,product,cancellation
0,K1,정상
1,K1,정상


In [43]:
df_TOCC_product.describe() 
# count의 값이 차이가 있다는 것은 null값이 존재한다는 뜻
# unique 를 확인한 값을 통해 범주형임을 알수 있음 

,product,cancellation
count,51303,51279
unique,6,2
top,K1,정상
freq,39134,50372


In [44]:
df_TOCC_product.isnull().sum()

product          1
cancellation    25
dtype: int64

In [45]:
df_TOCC_product = df_TOCC[['cancellation', 'product']].dropna().copy()
df_TOCC_product.describe() 

,cancellation,product
count,51278,51278
unique,2,6
top,정상,K1
freq,50371,39111


In [46]:
df_TOCC_product_pivot = pd.pivot_table(data=df_TOCC_product, index='cancellation', columns='product', aggfunc=len)

#### index,columns의 순서가 바뀌어도 p-value의 값은 동일 

In [47]:
## product
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True) 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

In [48]:
##  cancellation
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True) 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

#### 선정 변수 : state, cancellation
- 이런 변수 구성은 참조 되지 않는다.(명확하게 얘기하면 확인불가)
- 카이제곱을 할 수 없는 상황
- null값이 있으면 카이제곱이 안됨

In [49]:
df_TOCC_error = df_TOCC[['state', 'cancellation']]
#df_TOCC_error[:2]
df_TOCC_error.describe()

# count의 값이 차이가 있다는 것은 null값이 존재한다는 뜻
# unique 를 확인한 값을 통해 범주형임을 알수 있음 

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


In [50]:
df_TOCC_error.isnull().sum()

state            0
cancellation    25
dtype: int64

In [51]:
df_TOCC_error = df_TOCC[['state', 'cancellation']].dropna().copy()
df_TOCC_error.describe()

,state,cancellation
count,51279,51279
unique,4,2
top,계약확정,정상
freq,50326,50372


In [52]:
df_TOCC_error_pivot = pd.pivot_table(data=df_TOCC_error, index='state', columns='cancellation', aggfunc=len)
df_TOCC_error_pivot

cancellation,정상,해약
state,,
계약확정,50326.0,NaN
기간만료,32.0,NaN
해약진행중,14.0,NaN
해약확정,NaN,907.0


#### null값이 있으면 카이제곱이 안됨
- 해당 경우 : 귀무가설이 맞다 

In [53]:
stats.chi2_contingency(observed=df_TOCC_error_pivot, correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=3, expected_freq=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]]))